In [54]:
import pandas as pd
import numpy as np
import os
import time
import sys
import json
from random import randint
from sqlalchemy import create_engine
from datetime import datetime, timedelta
sys.path.append('../core')
from mysql import MYSQL
from negative_feature import NegativeSampler

In [3]:
pd.options.mode.chained_assignment = None
# db_connection = 'mysql+pymysql://douzi@traffic110:1qaz!QAZ2wsx@WSX@traffic110.mysql.database.azure.com/traffic'
db_connection = 'mysql+pymysql://root:@localhost/accident'
conn = create_engine(db_connection)
df = pd.read_sql("""
     SELECT * FROM static_feature
     """, conn)

In [4]:
df['target'] = 1
# df['accident_counts'] = 1
df['ACCIDENTTIME'] = pd.to_datetime(df['ACCIDENTTIME']).dt.strftime('%H:00:00')
df['hour'] = pd.to_datetime(df['ACCIDENTTIME'], format='%H:%M:%S').dt.hour
df['month'] = pd.to_datetime(df['ACCIDENTDATE'], format='%Y-%m-%d').dt.month
df['timestamp'] = pd.to_datetime(df['ACCIDENTDATE'] + ' ' + df['ACCIDENTTIME'].apply(str))
df['datetime'] = df['timestamp'].values.astype(np.int64) // 10 ** 9
df['segment_id'] = df['datetime'].apply(str) + df['Route_No']
df['ROAD_TYPE'] = df['ROAD_TYPE'].fillna('UNKNOWN')
df = df[['timestamp', 'ACCIDENTDATE', 'ACCIDENTTIME', 'Route_No', 'segment_id', 'hour', 'DAY_OF_WEEK', 'month', 'SPEED_ZONE', 
         'Light_Condition', 'ROAD_TYPE', 'DIRECTION_LOCATION', 'snowing', 'raining', 'foggy', 
         'smoke', 'dust', 'strong_winds', 'wind_dir', 'wind_speed', 'temperature', 
         'SURFACE_COND', 'NODE_TYPE','Deg_Urban_Name', 'target']]
# df.dropna(how='any')
# df.replace([np.inf, -np.inf], np.nan).dropna(axis=0)
df = df[pd.notnull(df['wind_dir'])]
df = df[pd.notnull(df['temperature'])]
# print(df.head())
print(df)

In [5]:
accident_counts = df.groupby('segment_id').size().reset_index(name='accident_counts')
print(accident_counts)

              segment_id  accident_counts
0       1136073600140545                1
1       1136073600150011                1
2       1136073600159568                1
3       1136073600191333                4
4       1136073600207419                1
5       1136073600226495                1
6         11360736002750                1
7       1136077200151629                1
8       1136077200182361                1
9       1136077200208930                1
10      1136077200279675                1
11        11360772002820                2
12        11360772002900                2
13        11360772005010                1
14      1136080800162379                1
15      1136080800276014                1
16        11360880002830                1
17        11360916005520                1
18      1136095200143495                1
19        11360952005500                1
20      1136102400109198                1
21        11361024002080                1
22      1136102400275502          

In [6]:
df = pd.merge(df, accident_counts, on=('segment_id'))
df = df.drop_duplicates('segment_id',keep='first',inplace=False)
df['segment_id'].apply(str)
# df.to_csv('./positive_feature.csv', index=False)
print(df.head())

            timestamp ACCIDENTDATE ACCIDENTTIME Route_No        segment_id  \
0 2006-02-23 15:00:00   2006-02-23     15:00:00   180994  1140706800180994   
1 2006-02-23 15:00:00   2006-02-23     15:00:00   213222  1140706800213222   
2 2006-02-06 21:00:00   2006-02-06     21:00:00     5044    11392596005044   
3 2006-02-06 21:00:00   2006-02-06     21:00:00   138932  1139259600138932   
4 2006-01-09 17:00:00   2006-01-09     17:00:00     5026    11368260005026   

   hour  DAY_OF_WEEK  month  SPEED_ZONE  Light_Condition       ...         \
0    15            5      2          60                1       ...          
1    15            5      2          60                1       ...          
2    21            2      2          60                3       ...          
3    21            2      2          60                3       ...          
4    17            2      1          60                1       ...          

  dust strong_winds  wind_dir  wind_speed  temperature  SURFACE_COND

## Weather Feature

In [8]:
df_weather = df[['ACCIDENTDATE','snowing', 'raining', 'foggy', 'smoke', 'dust', 'strong_winds',
                'wind_dir', 'wind_speed', 'temperature', 'SURFACE_COND']].drop_duplicates(subset=['ACCIDENTDATE'], keep='first')
df_weather.to_csv('../data/weather_feature.csv', index=False)
print(df_weather)

       ACCIDENTDATE  snowing  raining  foggy  smoke  dust  strong_winds  \
0        2006-02-23        0        0      0      0     0             0   
2        2006-02-06        0        0      0      0     0             0   
4        2006-01-09        0        0      0      0     0             0   
7        2006-01-24        0        0      0      0     0             0   
9        2006-02-24        0        0      0      0     0             0   
11       2006-01-17        0        0      0      0     0             0   
12       2006-03-05        0        0      0      0     0             0   
17       2006-02-25        0        0      0      0     0             0   
19       2006-02-16        0        0      0      0     0             0   
21       2006-03-10        0        0      0      0     0             0   
23       2006-02-17        0        0      0      0     0             0   
25       2006-01-13        0        0      0      0     0             0   
27       2006-01-31      

## Road Feature

In [16]:
df_road = df[['Route_No','SPEED_ZONE', 'Light_Condition', 'ROAD_TYPE', 'DIRECTION_LOCATION',
              'NODE_TYPE', 'Deg_Urban_Name']].drop_duplicates(subset=['Route_No'], keep='first')
df_road.to_csv('../data/road_feature.csv', index=False)
print(df_road.dtypes)


Route_No              object
SPEED_ZONE             int64
Light_Condition        int64
ROAD_TYPE             object
DIRECTION_LOCATION    object
NODE_TYPE             object
Deg_Urban_Name        object
dtype: object


## Negative Feature

In [57]:
df = pd.read_csv('../data/positive_feature.csv')
# df_sample = df[['timestamp', 'Route_No', 'segment_id', 'accident_counts']]

In [43]:
N = df.shape[0]*6
ns = NegativeSampler(N, df)
df_negative = ns.sample().reset_index()
print(df_negative)
# df_negative.to_csv('./negative_0.csv', index=False)

               segment_id           timestamp  Route_No  target  DAY_OF_WEEK  \
0        1250686800171145 2009-08-19 13:00:00    171145       0            3   
1        1221674400171228 2008-09-17 18:00:00    171228       0            3   
2        1334962800276721 2012-04-20 23:00:00    276721       0            5   
3          12422916005736 2009-05-14 09:00:00      5736       0            4   
4          13220460005057 2011-11-23 11:00:00      5057       0            3   
5        1164038400207353 2006-11-20 16:00:00    207353       0            1   
6          11714292005798 2007-02-14 05:00:00      5798       0            3   
7        1304668800104939 2011-05-06 08:00:00    104939       0            5   
8          12161700005970 2008-07-16 01:00:00      5970       0            3   
9        1374184800172484 2013-07-18 22:00:00    172484       0            4   
10         11888676002110 2007-09-04 01:00:00      2110       0            2   
11         11824200005733 2007-06-21 10:

In [47]:
df_negative['Route_No'] = df_negative['Route_No'].astype(str)
df_negative['target'] = 0
df_negative['accident_counts'] = 0
print(df_negative.dtypes)

segment_id                    object
timestamp             datetime64[ns]
Route_No                      object
target                         int64
DAY_OF_WEEK                    int64
month                          int64
hour                           int64
ACCIDENTDATE                  object
ACCIDENTTIME                  object
snowing                        int64
raining                        int64
foggy                          int64
smoke                          int64
dust                           int64
strong_winds                   int64
wind_dir                      object
wind_speed                   float64
temperature                  float64
SURFACE_COND                   int64
SPEED_ZONE                     int64
Light_Condition                int64
ROAD_TYPE                     object
DIRECTION_LOCATION            object
NODE_TYPE                     object
Deg_Urban_Name                object
accident_counts                int64
dtype: object


In [45]:
df_negative['ACCIDENTDATE'] = df_negative['timestamp'].dt.strftime('%Y-%m-%d')
df_negative['ACCIDENTTIME'] = df_negative['timestamp'].dt.strftime('%H:%M:%S')
df_negative = pd.merge(df_negative, df_weather, on=('ACCIDENTDATE'))
df_negative = pd.merge(df_negative, df_road, on=('Route_No'))

In [48]:
df_negative = df_negative[['timestamp', 'ACCIDENTDATE', 'ACCIDENTTIME', 'Route_No', 
                          'segment_id', 'hour', 'DAY_OF_WEEK', 'month', 'SPEED_ZONE',
                          'Light_Condition', 'ROAD_TYPE', 'DIRECTION_LOCATION', 'snowing',
                          'raining', 'foggy', 'smoke', 'dust', 'strong_winds', 'wind_dir',
                          'wind_speed', 'temperature', 'SURFACE_COND', 'NODE_TYPE',
                          'Deg_Urban_Name', 'target', 'accident_counts']]
df_negative.to_csv('../data/negative_feature.csv', index=False)
print(df_negative)

                 timestamp ACCIDENTDATE ACCIDENTTIME Route_No  \
0      2009-08-19 13:00:00   2009-08-19     13:00:00   171145   
1      2009-08-19 13:00:00   2009-08-19     13:00:00   171145   
2      2016-12-10 07:00:00   2016-12-10     07:00:00   171145   
3      2009-11-15 10:00:00   2009-11-15     10:00:00   171145   
4      2016-04-08 01:00:00   2016-04-08     01:00:00   171145   
5      2017-07-17 02:00:00   2017-07-17     02:00:00   171145   
6      2013-02-26 09:00:00   2013-02-26     09:00:00   171145   
7      2017-11-10 21:00:00   2017-11-10     21:00:00   171145   
8      2017-01-01 02:00:00   2017-01-01     02:00:00   171145   
9      2014-12-18 18:00:00   2014-12-18     18:00:00   171145   
10     2007-08-02 09:00:00   2007-08-02     09:00:00   171145   
11     2009-04-12 10:00:00   2009-04-12     10:00:00   171145   
12     2013-05-15 11:00:00   2013-05-15     11:00:00   171145   
13     2016-02-26 01:00:00   2016-02-26     01:00:00   171145   
14     2009-01-23 10:00:0

In [58]:
print(df)

                  timestamp ACCIDENTDATE ACCIDENTTIME  Route_No  \
0       2006-02-23 15:00:00   2006-02-23     15:00:00    180994   
1       2006-02-23 15:00:00   2006-02-23     15:00:00    213222   
2       2006-02-06 21:00:00   2006-02-06     21:00:00      5044   
3       2006-02-06 21:00:00   2006-02-06     21:00:00    138932   
4       2006-01-09 17:00:00   2006-01-09     17:00:00      5026   
5       2006-01-09 17:00:00   2006-01-09     17:00:00      5044   
6       2006-01-09 17:00:00   2006-01-09     17:00:00    150347   
7       2006-01-24 17:00:00   2006-01-24     17:00:00      2090   
8       2006-01-24 17:00:00   2006-01-24     17:00:00    159508   
9       2006-02-24 07:00:00   2006-02-24     07:00:00      2460   
10      2006-02-24 07:00:00   2006-02-24     07:00:00    105177   
11      2006-01-17 08:00:00   2006-01-17     08:00:00      5127   
12      2006-03-05 21:00:00   2006-03-05     21:00:00      5127   
13      2006-01-17 08:00:00   2006-01-17     08:00:00      587

In [59]:
df_train = pd.concat([df, df_negative])
print(df_train)

                  timestamp ACCIDENTDATE ACCIDENTTIME Route_No  \
0       2006-02-23 15:00:00   2006-02-23     15:00:00   180994   
1       2006-02-23 15:00:00   2006-02-23     15:00:00   213222   
2       2006-02-06 21:00:00   2006-02-06     21:00:00     5044   
3       2006-02-06 21:00:00   2006-02-06     21:00:00   138932   
4       2006-01-09 17:00:00   2006-01-09     17:00:00     5026   
5       2006-01-09 17:00:00   2006-01-09     17:00:00     5044   
6       2006-01-09 17:00:00   2006-01-09     17:00:00   150347   
7       2006-01-24 17:00:00   2006-01-24     17:00:00     2090   
8       2006-01-24 17:00:00   2006-01-24     17:00:00   159508   
9       2006-02-24 07:00:00   2006-02-24     07:00:00     2460   
10      2006-02-24 07:00:00   2006-02-24     07:00:00   105177   
11      2006-01-17 08:00:00   2006-01-17     08:00:00     5127   
12      2006-03-05 21:00:00   2006-03-05     21:00:00     5127   
13      2006-01-17 08:00:00   2006-01-17     08:00:00     5871   
14      20

In [60]:
df_train = df_train.dropna(how='any',axis=0)
df_train.to_csv('../data/final_data.csv', index=False)

In [61]:
print(df_train.shape[0])

985264
